In [1]:
import torch
from torch.utils.data import DataLoader
from models.resnet_clothing_model import ClothingClassifier
from data.dataset import TrainGarmentDataset
from utils.loss import compute_loss
from utils.metrics import accuracy
from data.dataset import TrainGarmentDataset
from data.transformation import CustomResNetTransform
from tqdm import tqdm

/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# 1) Prepare Data
train_dataset = TrainGarmentDataset(transform=CustomResNetTransform())
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)

In [3]:
# 2) Construct loss and optimizer
NUM_CLASSES = 50
model = ClothingClassifier(num_classes=NUM_CLASSES)
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet50_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet50_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


In [4]:
# 3) Training loop
num_epochs = 10

for epoch in range(num_epochs):
    model.train()  # Set model to training mode
    running_loss = 0.0
    total_correct = 0
    total_samples = 0
    
    # Iterate over training data
    with tqdm(train_loader, desc=f"Epoch {epoch+1}/{num_epochs}", unit="batch") as pbar:
        for images, category_id in pbar:
            # Zero the gradients
            optimizer.zero_grad()
            
            # Forward pass
            preds = model(images)
            
            # Compute loss: pass the predictions and target as a dict
            loss = compute_loss(preds, {'category_id': category_id})
            
            # Backward pass and optimization
            loss.backward()
            optimizer.step()
            
            # Update running loss
            running_loss += loss.item()
            
            # Update accuracy: accuracy() should return the number of correct predictions for the batch
            batch_correct = accuracy(preds, category_id)
            total_correct += batch_correct
            total_samples += images.size(0)
            
            # Update progress bar
            pbar.set_postfix(loss=loss.item())
    
    # Compute epoch loss and accuracy
    epoch_loss = running_loss / len(train_loader)
    epoch_accuracy = (total_correct / total_samples) * 100
    print(f"Epoch {epoch+1}/{num_epochs}: Loss = {epoch_loss:.4f}, Accuracy = {epoch_accuracy:.2f}%")

# Optionally, save the model after training:
torch.save(model.state_dict(), 'clothing_classifier.pth')


Epoch 1/10:  16%|█▌        | 69/438 [08:41<44:29,  7.23s/batch, loss=2.71]  

In [ ]:
print("Training complete. Saving model...")
torch.save(model.state_dict(), 'clothing_classifier.pth')
print("Model saved.")

Training complete. Saving model...
Model saved.
